**Simple example for Asg1 with handcrafted feature**

An example diagram for instance search.
Please read the slide "Information for Asg1" first.

In every bounding box file (the txt files), each line records the coordinates of one bounding box in format: x of top-left point, y of top-left point, width, height.

The bounding box information is only provided for the query images.

Notes: it's possible that there are more than one instances in a query image (see the annotation(s) in the corresponding txt file), you should find the most similar image rank list in the gallery by considering all the instances.

In [222]:
import numpy as np 
import cv2 
import pandas as pd
import glob
import time 
import matplotlib.pyplot as plt
from tqdm import tqdm


 Download the dataset from this linke [onedrive link](https://portland-my.sharepoint.com/:u:/g/personal/srwang3-c_my_cityu_edu_hk/EZ0BIZatMIJMoiCG4-uy6okBSlXLJD7TUyIDG1lbKUJ0eA?e=lbKz4a).
And unzip the downloaded file into some path. 
> In this tutorial, the path is `/Users/txsing/datasets_4186`, please replace to your own accordingly.

Initialize the necessary parameters, including paths, feature extractors.

In [223]:
download_path='D:/Users/KLIVANCHAN/Desktop/CityU' # change to your own download path
path_query=download_path+'/query_4186'
path_query_txt=download_path+'/query_txt_4186'

# path_query_txt is the directory to the bounding box information of the instance(s) for the query images
path_gallery=download_path+'/gallery_4186'

name_query=glob.glob(path_query+'/*.jpg')
num_query=len(name_query)
name_gallery=glob.glob(path_gallery+'/*.jpg')
num_gallery=len(name_gallery)

sift = cv2.ORB_create()
# bf = cv2.BFMatcher()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
# search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,{})

record_all=np.zeros((num_query,len(name_gallery)))

query_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_query+'/*.jpg')]
query_imgs_no = [x[:-4] for x in query_imgs_no]

gallery_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_gallery+'/*.jpg')]
gallery_imgs_no = [x[:-4] for x in gallery_imgs_no]

Process

Feature extraction -> calculate distance -> compare

In [224]:
# the iteration loop for query 
# iteration times is 2 here, only for the demonstration
for i, query_img_no in enumerate(query_imgs_no[0:1]):
    
    time_s = time.time()
    dist_record=[]
    per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
    per_query=cv2.imread(per_query_name)
    gallery_imgs_no_desc=[]
    
    # read boundary from text file
    queryfilename = path_query_txt+'/'+str(query_img_no)+'.txt'
    boundary_file = open(queryfilename, 'r')
    boundary = boundary_file.readline().strip().split(' ')
    boundary = [int(b) for b in boundary]
    boundary_file.close()
    
    # crop the image
    x ,y, w, h = boundary
    query_boundary = per_query[y:y+h, x:x+w]
    
    # feature extraction for per query
    # the bounding box information is not considered
    # quite naive, just an example
    per_query_kp, per_query_des = sift.detectAndCompute(query_boundary,None)
    
    # the iteration loop for gallery
    for j, gallery_img_no in tqdm(enumerate(gallery_imgs_no), desc=f"Processing query part {i}"):
        per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
        per_gallery=cv2.imread(per_gallery_name)
        
        # feature extraction for per gallery
        per_gallery_kp, per_gallery_des = sift.detectAndCompute(per_gallery,None)
        
        # use part of the features to make the calculation feasible
        # quite naive, just an example
#         matches = bf.knnMatch(np.asarray(per_query_des,np.float32),np.asarray(per_gallery_des,np.float32),k=2)
        matches = flann.knnMatch(per_query_des.astype(np.float32),per_gallery_des.astype(np.float32),k=2)
    
        # Compute similarity score for each match
        sim_scores = []
        for match in matches:
            if len(match) == 2:
                (m,n) = match
                query_desc = per_query_des[m.queryIdx]
                gallery_desc = per_gallery_des[m.trainIdx]
                sim_score = np.sum((np.double(query_desc)-np.double(gallery_desc))**2)/np.prod(np.shape(query_desc))
                if np.isnan(sim_score):  # Check if similarity score is NaN
                    continue
                sim_scores.append(sim_score)

        # Compute average similarity score for all matches
        avg_sim_score = np.mean(sim_scores)
        dist_record.append(avg_sim_score)
#         cv2.imshow('result',per_result_img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

    # find the indexes with descending similarity order
    ascend_index=sorted(range(len(dist_record)), key=lambda k: dist_record[k])
    # update the results for one query
    for k in range(len(ascend_index)):
        gallery_imgs_no_desc.append(np.array(gallery_imgs_no)[ascend_index[k]])
    record_all[i,:]= gallery_imgs_no_desc
    time_e = time.time()
    sorted(dist_record)
    print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))
    query_idx = i
    print(f'For query image No. {query_imgs_no[query_idx]}, the top ranked similar image No. is {gallery_imgs_no_desc[0]}.')

Processing query part 0: 4it [00:00, 29.57it/s]

0 4122.573625283447
1 4088.8088151927436
10 4135.047689909297
100 4271.086380385487
1000 4049.517431972789
1001 4121.810445011338
1002 4080.474560657596


Processing query part 0: 12it [00:00, 30.27it/s]

1003 4283.208333333333
1004 3882.876558956916
1005 4064.795493197279
1006 4343.199263038548
1007 4005.770975056689
1008 3855.732001133787
1009 4087.6639739229026


Processing query part 0: 20it [00:00, 33.10it/s]

...

986 4145.747023809524
987 4083.2412840136053
988 4149.24929138322
989 4109.0856717687075
99 3831.0717120181407
990 4185.302721088436
991 4140.657879818594


Processing query part 0: 4997it [02:32, 33.46it/s]

992 4023.876842403628
993 4000.7379535147393
994 3921.0257936507937
995 3903.8073270975055
996 4098.181972789115
997 4524.84587585034
998 4241.294997165533


Processing query part 0: 5000it [02:32, 32.69it/s]


999 4303.707341269841
retrieval time for query 2714 is 156.60904812812805s
For query image No. 2714, the top ranked similar image No. is 4256.


Output 

Save the rank list file.

In [225]:
# write the output file following the example
f=open(r'./rank_list.txt','w')
for i in range(num_query):
    f.write('Q'+str(i+1)+': ')
    for j in range(len(name_gallery)):
        f.write(str(np.int32(record_all[i,j]))+' ')
    f.write('\n')
f.close()